### <b>Installation for the StyleGAN Encoder Network</b>

* Download the pixel2style2pixel (pSp) source code.

<pre>
!git clone https://github.com/eladrich/pixel2style2pixel.git pixel2style2pixel
</pre>

* Ninja is required to load c++ extensions in the pSp encoder.

<pre>
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force
</pre>

* Download the pre-trained pSp model.

<pre>
import os

# Make a folder that contains pre-trained models
save_path = "./pretrained_models"
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Download the pre-trained pSp model.
file_path = os.path.join(save_path, "psp_ffhq_encode.pt")
!wget https://postechackr-my.sharepoint.com/:u:/g/personal/dongbinna_postech_ac_kr/EZDa4b7PITZCqrCcVVFrN4UBrFYkZmENV0uFQf1fE5gR5Q?download=1 -O $file_path
</pre>

* (Option) If the libcudart.so is not found, you may have to set environment variables.
    * Then, you may have to run the jupyter-notebook with "allow-root" option.

<pre>
# The example script:
export PATH=$PATH:/usr/local/cuda-10.0
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda-10.0/lib64
</pre>

* Download the face image alignment model.

<pre>
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2
</pre>

* Download example images for the inversion.

<pre>
!wget https://postechackr-my.sharepoint.com/:u:/g/personal/dongbinna_postech_ac_kr/ETkdz1PCJ_NAtj4NUdiQCVsBv2UpVamnhm-MUcD59NN-GA?download=1 -O inversion_images.zip
!unzip inversion_images.zip
</pre>

### <b>Installation for the Classification Network</b>

* Download the classification model.

<pre>
!wget https://postechackr-my.sharepoint.com/:u:/g/personal/dongbinna_postech_ac_kr/EX15A0wm8MBLrBsT-9ARA-gBZ6W-RwmSw1IgYZzan4dELg?download=1 -O facial_identity_classification_using_transfer_learning_with_ResNet18_resolution_256_normalize_05.pth
</pre>

* Download the Celeb-HQ facial identity recognition dataset.

<pre>
%%capture
!wget https://postechackr-my.sharepoint.com/:u:/g/personal/dongbinna_postech_ac_kr/EcWlrJ2qjtRJtQVIwR1kW2EBTj8le3PdXI4TYsKErsGMcA?download=1 -O CelebA_HQ_facial_identity_dataset_test.zip
!unzip CelebA_HQ_facial_identity_dataset_test.zip -d ./CelebA_HQ_facial_identity_dataset_test
</pre>

* Download the corrected facial identity dataset.

<pre>
%%capture
!wget https://postechackr-my.sharepoint.com/:u:/g/personal/dongbinna_postech_ac_kr/EQIF7ZqRxDJCjTwWiO1xPe4BqpenC93AEpTnRpSOlrPl5g?download=1 -O corrected_facial_identity_images_ResNet18.zip
!unzip corrected_facial_identity_images_ResNet18.zip -d ./corrected_facial_identity_images_ResNet18
</pre>

### <b>Installation for Computing Metrics</b>

* Download the pre-trained CurricularFace model.

<pre>
import os

# Make a folder that contains pre-trained models
save_path = "./pretrained_models"
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Download the pre-trained CurricularFace model.
file_path = os.path.join(save_path, "CurricularFace_Backbone.pth")
!wget https://postechackr-my.sharepoint.com/:u:/g/personal/dongbinna_postech_ac_kr/Eaaz-dJcRj1GspN39J3-5GkBbzkOZL6dvXD2MsVwOvwQsg?download=1 -O $file_path
</pre>

* Download the pre-trained MTCNN model.

<pre>
import os

# Make a folder that contains pre-trained models
save_path = "./pretrained_models"
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Download the pre-trained MTCNN model.
file_path = os.path.join(save_path, "mtcnn.tar.gz")
!wget https://postechackr-my.sharepoint.com/:u:/g/personal/dongbinna_postech_ac_kr/EdSEMj6JF7VEpXo296X6fekB0uvcyMRH8LkUj15VafAuJA?download=1 -O $file_path
!tar -zxvf $file_path -C $save_path
</pre>

In [ ]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

from argparse import Namespace
import time
import sys
import pprint
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import display
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms

sys.path.append("./pixel2style2pixel")

import dlib
from scripts.align_all_parallel import align_face
from datasets import augmentations
from utils.common import tensor2im, log_input_image

from models.mtcnn.mtcnn import MTCNN
from models.encoders.model_irse import IR_101
from configs.paths_config import model_paths
from criteria.lpips.lpips import LPIPS

In [ ]:
# Facial similarity score
facenet = IR_101(input_size=112)
facenet.load_state_dict(torch.load("./pretrained_models/CurricularFace_Backbone.pth"))
facenet.cuda()
facenet.eval()
mtcnn = MTCNN()

# LPIPS loss
lpips_loss = LPIPS(net_type='alex')

In [ ]:
img_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

def image_loader(image_name):
    """load image, returns cuda tensor"""
    image = Image.open(image_name)
    image = img_transforms(image)
    image = image.unsqueeze(0)
    return image.cuda()

### Evaluate the attack results

In [ ]:
root = ATTACK_RESULT_FOLDER

In [ ]:
import os

# count all directories in which 20000 queries are conducted
filenames = os.listdir(root)
all_directory = []
for filename in filenames:
    log_file = os.path.join(root, filename, 'log')
    with open(log_file, 'r') as f:
        log = f.readlines()
        final_line = log[-1].strip()
        if final_line.split(' ')[0] == '20000':
            all_directory.append(filename)

length = len(all_directory)
print(length, 'logs are available')

all_mses = []
queries = []
all_simils = []
all_lpipses = []

cnt = 0
for directory in all_directory:
    log_file = os.path.join(root, directory, 'log')
    with open(log_file, 'r') as f:
        log = f.readlines()
    mses = []
    log_length = len(log)
    available = log[len(log) - 12:]
    source, target = log[0].split()
    for av in available:
        query, mse, linf, simil, lpips = av.strip().split()
        mses.append(mse)
        queries.append(query)
    all_mses.append(mses)

    simils = []
    lpipses = []
    for i in [100, 200, 300, 500, 1000, 2000, 3000, 5000, 8000, 10000, 15000, 20000]:
        current_target = image_loader(os.path.join(root, directory, str(i) + '.png'))
        source = image_loader(os.path.join(root, directory, 'encoded_source.png'))

        t = facenet(current_target)
        s = facenet(source)
        simil = torch.bmm(t.unsqueeze(1), s.unsqueeze(2))
        simils.append(simil[0][0].item()) # similarity score
        lpipses.append(lpips_loss(current_target, source).item()) # LPIPS score
    
    all_simils.append(simils)
    all_lpipses.append(lpipses)
    cnt +=1
    print(cnt)


for i in range(12):
    summary_mse = 0
    for m in all_mses:
        summary_mse += float(m[i])
    summary_simil = 0
    for m in all_simils:
        summary_simil += m[i]
    summary_lpips = 0
    for m in all_lpipses:
        summary_lpips += m[i]
    print(queries[i], summary_mse / len(all_mses), summary_simil / len(all_mses), summary_lpips / len(all_mses))

### Evaluate the real-world attack results

In [ ]:
attack_target = ATTACK_RESULT
attack_source = SOURCE_IMAGE

In [ ]:
current_target = image_loader(attack_target)
source = image_loader(attack_source)

t = facenet(current_target)
s = facenet(source)
simil = torch.bmm(t.unsqueeze(1), s.unsqueeze(2))
print(simil[0][0].item())
print(lpips_loss(current_target, source).item())